In [35]:
# !pip install pandas
# !pip install nltk
!pip install fasttext

  Using cached fasttext-0.9.2.tar.gz (68 kB)
Using legacy 'setup.py install' for fasttext, since package 'wheel' is not installed.
    Running setup.py install for fasttext ... done
You should consider upgrading via the '/home/gitpod/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
import os
import argparse
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import csv
import nltk
from sklearn.model_selection import train_test_split

In [5]:
stemmer = nltk.stem.PorterStemmer()

categories_file_name = r'/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml'

queries_file_name = r'/workspace/datasets/train.csv'
output_file_name = r'/workspace/datasets/labeled_query_data.txt'
output_train_file_name = r'/workspace/datasets/train'
output_test_file_name = r'/workspace/datasets/test'


# parser = argparse.ArgumentParser(description='Process arguments.')
# general = parser.add_argument_group("general")
# general.add_argument("--min_queries", default=1,  help="The minimum number of queries per category label (default is 1)")
# general.add_argument("--output", default=output_file_name, help="the file to output to")

# args = parser.parse_args()
# output_file_name = args.output

# if args.min_queries:
#     min_queries = int(args.min_queries)

# The root category, named Best Buy with id cat00000, doesn't have a parent.
root_category_id = 'cat00000'

tree = ET.parse(categories_file_name)
root = tree.getroot()

# Parse the category XML file to map each category id to its parent category id in a dataframe.
categories = []
parents = []
for child in root:
    id = child.find('id').text
    cat_path = child.find('path')
    cat_path_ids = [cat.find('id').text for cat in cat_path]
    leaf_id = cat_path_ids[-1]
    if leaf_id != root_category_id:
        categories.append(leaf_id)
        parents.append(cat_path_ids[-2])
parents_df = pd.DataFrame(list(zip(categories, parents)), columns =['category', 'parent'])
df = pd.read_csv(queries_file_name)[['category', 'query']]
df = df[df['category'].isin(categories)]
# IMPLEMENT ME: Convert queries to lowercase, and optionally implement other normalization, like stemming.
df["query"] =  df["query"].str.lower()

Try converting the queries to lowercase, stripping quotation marks, and removing any other punctuation or unusual characters. Treat anything that’s not a number or letter as a space, and trim multiple spaces to a single space. Optionally, use the nltk stemmer to stem the queries. If you see any other opportunities to normalize the queries, go for it!

In [6]:
regexes = [(r"[\+]", " plus"),
(r"\$(\d+)", r"\1 dollars"),
(r"\&", " and "),
(r"\s*[\./_-]\s*", " "),
(r"[^\w\s]", "")]

for regex, replace in regexes:
    df["query"] = df["query"].str.replace(regex, replace, regex = True)

df.head()

,category,query
0,abcat0101001,televisiones panasonic 50 pulgadas
1,abcat0101001,sharp
2,pcmcat193100050014,nook
3,abcat0101001,rca
4,abcat0101005,rca


In [7]:
df["query"] = df["query"].apply(lambda x: stemmer.stem(x))

KeyboardInterrupt: 

In [9]:
parents_dict = parents_df.set_index("category")["parent"]
parents_dict["cat00000"] = "cat00000"
parents_dict

category
abcat0010000                  cat00000
abcat0011000              abcat0010000
abcat0011001              abcat0011000
abcat0011002              abcat0011000
abcat0011003              abcat0011000
                           ...        
pcmcat97200050015             cat15063
pcmcat99000050001    pcmcat50000050006
pcmcat99000050002    pcmcat99000050001
pcmcat99300050000             cat15063
cat00000                      cat00000
Name: parent, Length: 4640, dtype: object

In [10]:
category_cutoff_count = 10000

def categories_below_cutoff():
    return (df["category"].value_counts()<category_cutoff_count).sum()

num_of_categories_below_cutoff = categories_below_cutoff()
while num_of_categories_below_cutoff > 0:
    categories_below = df["category"].value_counts().where(lambda x: x<category_cutoff_count).dropna()
    df["category"] = df["category"].apply(lambda x: parents_dict[x] if x in categories_below else x)
    num_of_categories_below_cutoff = categories_below_cutoff()



In [12]:
df['label'] = '__label__' + df['category']

# Output labeled query data as a space-separated file, making sure that every category is in the taxonomy.
df = df[df['category'].isin(categories)]
df['output'] = df['label'] + ' ' + df['query']

In [29]:
train = df[['output']].sample(frac = 0.8)
test = df[["output"]].drop(train.index)

In [33]:
df["label"].value_counts()

__label__cat02015              177638
__label__abcat0101001           80213
__label__pcmcat247400050000     79245
__label__pcmcat209000050008     74258
__label__abcat0900000           69044
                                ...  
__label__abcat0703001           10400
__label__abcat0301014           10387
__label__pcmcat164200050013     10187
__label__abcat0715001           10144
__label__pcmcat186100050006     10051
Name: label, Length: 69, dtype: int64

In [37]:
import fasttext
model = fasttext.load_model("/workspace/search_with_machine_learning_course/week3/model2.bin")


(('__label__abcat0101001',
  '__label__pcmcat200900050015',
  '__label__abcat0106004'),
 array([0.98524475, 0.00435535, 0.00350147]))

In [52]:
categories, probs = model.predict('car', k = 69)
#### W3: create filters and boosts
threshold = 0.5
for i in range(69):
    if sum(probs[0:i])> threshold:
        categories = categories[0:i]
        break
" OR ".join(categories)

'__label__cat09000 OR __label__pcmcat143700050032 OR __label__pcmcat219900050000 OR __label__cat02015 OR __label__abcat0500000 OR __label__abcat0301014 OR __label__abcat0201007'

In [48]:
a

('__label__abcat0101001',
 '__label__pcmcat200900050015',
 '__label__abcat0106004',
 '__label__abcat0101005',
 '__label__cat02015',
 '__label__abcat0100000',
 '__label__abcat0106005',
 '__label__pcmcat143200050016',
 '__label__pcmcat233200050010',
 '__label__pcmcat200900050014',
 '__label__abcat0107020',
 '__label__abcat0303000',
 '__label__pcmcat200900050008',
 '__label__abcat0102008',
 '__label__abcat0500000',
 '__label__pcmcat158900050018',
 '__label__pcmcat143700050048',
 '__label__pcmcat128700050041',
 '__label__abcat0200000',
 '__label__pcmcat158400050073',
 '__label__abcat0106015',
 '__label__abcat0106001',
 '__label__abcat0102003',
 '__label__pcmcat186100050006',
 '__label__abcat0805000',
 '__label__pcmcat144700050004',
 '__label__abcat0102005',
 '__label__abcat0515004',
 '__label__abcat0201011',
 '__label__pcmcat242800050021',
 '__label__abcat0106016',
 '__label__abcat0403004',
 '__label__abcat0107040',
 '__label__pcmcat206100050021',
 '__label__pcmcat208100050009',
 '__label_